<a href="https://colab.research.google.com/github/TimothyLMoore/FCC_Neural-Network-SMS-Text-Classifier/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv('train-data.tsv',sep='\t', header=None, names=['class', 'text'])
valid_data = pd.read_csv('valid-data.tsv',sep='\t', header=None, names=['class', 'text'])

In [ ]:
train_data.head()

In [ ]:
train_data["class"] = train_data["class"].map({'ham': 0,'spam':1})
valid_data["class"] = valid_data["class"].map({'ham': 0,'spam':1})

In [ ]:
y_train = train_data.pop("class")
y_valid = valid_data.pop("class")
X_train = train_data.values.flatten()
X_valid = valid_data.values.flatten()

In [ ]:
from keras.layers import Dense, Embedding, Bidirectional, TextVectorization, LSTM
from keras import Sequential
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam

encoder = TextVectorization()
encoder.adapt(X_train)
input_size = len(encoder.get_vocabulary())

model = Sequential()
model.add(encoder)
model.add(Embedding(input_size, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(loss=BinaryCrossentropy(from_logits=True),
              optimizer=Adam(0.001),
              metrics=['accuracy']) #compiling the model

In [ ]:
history = model.fit(X_train,y_train, epochs=5, batch_size=64,
                    validation_data=(X_valid, y_valid))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    prediction = []

    pred = model.predict([pred_text])
    prediction.append(pred[0][0])
    if pred < 0.5:
      prediction.append('ham')
    else:
      prediction.append('spam')


    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
